In [5]:
ls -lh data/

Invalid switch - "".


# Loading The CSV File

In [3]:
%%time

import pandas as pd
df = pd.read_csv('data/fixed.csv')

Wall time: 13.9 s


In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2323984 entries, 0 to 2323983
Data columns (total 14 columns):
0                                     int64
 14                                   int64
 329                                  int64
 0                                    int64
 362                                  int64
 0.1                                  int64
 0.2                                  int64
 0.3                                  int64
 5                                    int64
 14-16\20170817012709_100.h264.txt    object
 2017-08-17 01:27:09                  object
 01:27:09                             object
 0:00:00                              object
 0.0604166666667                      float64
dtypes: float64(1), int64(9), object(4)
memory usage: 591.7 MB


In [10]:
df.describe()

,0,14,329,0,362,0.1,0.2,0.3,5,0.0604166666667
count,2.323984e+06,2.323984e+06,2.323984e+06,2.323984e+06,2.323984e+06,2.323984e+06,2323984.0,2323984.0,2.323984e+06,2.323984e+06
mean,8.271849e+03,2.762049e+02,1.465740e+02,5.330109e+01,4.196587e+07,1.182565e+02,0.0,0.0,8.043612e+00,5.222413e-01
std,5.161792e+03,1.625547e+02,7.780712e+01,9.297647e+01,3.404631e+07,1.532162e+02,0.0,0.0,1.431355e+01,2.707996e-01
min,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,3.620000e+02,0.000000e+00,0.0,0.0,1.000000e+00,0.000000e+00
25%,3.719000e+03,1.460000e+02,9.400000e+01,1.200000e+01,8.056592e+06,1.700000e+01,0.0,0.0,2.000000e+00,3.541667e-01
50%,8.098000e+03,2.720000e+02,1.310000e+02,2.700000e+01,4.154462e+07,6.000000e+01,0.0,0.0,3.000000e+00,5.333333e-01
75%,1.250700e+04,4.050000e+02,2.030000e+02,5.100000e+01,5.209650e+07,1.650000e+02,0.0,0.0,6.000000e+00,7.270833e-01
max,1.847600e+04,6.370000e+02,3.570000e+02,6.430000e+02,1.265195e+08,1.977000e+03,0.0,0.0,1.150000e+02,9.993056e-01


In [11]:
df.columns = ['frame', 'x', 'y', 'obj', 'size', 'seq', 'tbd1', 'tbd2','tbd3', 'filename', 'start', 'path_time', 'delta_time','tbd4']

In [12]:
df.head()

,frame,x,y,obj,size,seq,tbd1,tbd2,tbd3,filename,start,path_time,delta_time,tbd4
0,0,4,291,1,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
1,0,629,7,2,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
2,0,253,4,3,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
3,0,185,4,4,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
4,1700,5,102,5,362,0,0,0,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:28:17,0:01:08,0.061111


In [4]:
%%time
df = pd.read_csv('data/fixed.csv',usecols=[0,1,2,3,4,5,9,10,11,12])
df.columns = ['frame', 'x', 'y', 'obj', 'size', 'seq', 'filename', 'start', 'path_time', 'delta_time']


Wall time: 8.64 s


In [38]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2323984 entries, 0 to 2323983
Data columns (total 10 columns):
frame         int64
x             int64
y             int64
obj           int64
size          int64
seq           int64
filename      object
start         object
path_time     object
delta_time    object
dtypes: int64(6), object(4)
memory usage: 520.7 MB


In [5]:
%%time

df_int = df.select_dtypes(include=['int64'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')
                             
df[converted_int.columns] = converted_int
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2323984 entries, 0 to 2323983
Data columns (total 10 columns):
frame         uint16
x             uint16
y             uint16
obj           uint16
size          uint32
seq           uint16
filename      object
start         object
path_time     object
delta_time    object
dtypes: object(4), uint16(5), uint32(1)
memory usage: 445.4 MB
Wall time: 2.79 s


In [8]:
%%time
df_obj = df.select_dtypes(include = ['object']).copy()
converted_obj = pd.DataFrame()

for col in df_obj.columns:
       num_unique_values = len(df_obj[col].unique())
       num_total_values = len(df_obj[col])
       if num_unique_values / num_total_values < 0.5:
           converted_obj.loc[:,col] = df_obj[col].astype('category')
       else:
           converted_obj.loc[:,col] = df_obj[col]

df[converted_obj.columns] = converted_obj

Wall time: 31.9 ms


In [9]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2323984 entries, 0 to 2323983
Data columns (total 10 columns):
frame         uint16
x             uint16
y             uint16
obj           uint16
size          uint32
seq           uint16
filename      category
start         category
path_time     category
delta_time    category
dtypes: category(4), uint16(5), uint32(1)
memory usage: 55.5 MB
